In [2]:
import pandas as pd
import numpy as np
# from tqdm import tqdm
# from tqdm import tnrange, tqdm_notebook
# from time import sleep
import scipy
import operator
from IPython.core.display import display,HTML
from tqdm import tnrange, tqdm_notebook

try:
    import cPickle as pickle     #it is faster than pickle!
except:
    import pickle

In [3]:
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser

In [20]:
######### code to reformat the WoS text file (where all columns are STRINGS, since it is a cvs filem, even if they look like lists...!), 
##    and create a pickled dataframe for later merging with linkedin data


fields = ['author_id','year','author_name','affiliation','total_pubs']  # i only upload certain fields
#fields = ['author_id','uid','seq','year','author_name','affiliation','total_pubs']


#path_wos_win='C:\\Users\\julietta\\Work\\Dropbox_studies\\Data\\WoS_data\\Disambiguated_authors\\'
path_wos_linux='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/WoS_data/Disambiguated_authors/'

#%time df_disamb_wos_test=pd.read_csv(path_wos_linux+'only_USA_active_years_2000-2015_filtered_WoS_file.tsv', usecols=fields, sep="\t")#, nrows=100000)
%time df_disamb_wos_test=pd.read_csv(path_wos_linux+'only_USA_active_years_2005-2015_filtered_WoS_file_simplified.tsv', usecols=fields, sep="\t")#, nrows=100000)

df_disamb_wos_test.rename(columns={'affiliation':'affiliations'}, inplace=True) 
df_disamb_wos_test.rename(columns={'author_name':'author_names'}, inplace=True) 
#df_disamb_wos_test.rename(columns={'seq':'seqs'}, inplace=True) 


# fields = ['author_id','uid','seq','year','author_name','affiliation','total_pubs']

# df = pd.read_csv('data.csv', skipinitialspace=True, usecols=fields)

df_disamb_wos_test.head(3)

CPU times: user 1min 23s, sys: 4.25 s, total: 1min 27s
Wall time: 2min 20s


,author_id,year,author_names,affiliations,total_pubs
0,14972627,1973|1980|1983|1992|1993|1994|1995|1996|2000|2...,"Rossi, Adriano G.|Rossi, Adriano|Rossi, A. G.|...","Univ Milan, I-20122 Milan, Italy|OSPED CAMPOSA...",116
1,8198127,1971|1973|1987|1989|1990|1991|1993|1994|1995|1...,"Johnson, TM|JOHNSON, TM|Johnson, T|JOHNSON, T|...","UTAH STATE UNIV,DEPT BIOL,LOGAN,UT 84322|UNIV ...",24
2,879592,2014|2015,"Barker, J. G.|Barker, John G.","NIST, NIST Ctr Neutron Res, Gaithersburg, MD 2...",2


In [21]:
# df_disamb_wos_test.author_names.iloc[0]
#df_disamb_wos_test.shape

(6457847, 5)

In [27]:

def get_department_names(string):
    
    #raw_input()
   #print "\n\nORIGINAL:",string  
    

        #example of complete affiliation for one author
        ####   || separates papers, | separates multiple affil for the same author in a paper, _?_ means affill not sure (ambiguity)
         ### there are as many affill (sep by ||) as unique affil for an author (not necesarily as many as papers!!!)
        
    # #         Univ Edinburgh, Queens Med Res Inst, Ctr Inflammat Res, MRC, Edinburgh EH16 4TJ, Midlothian, Scotland||
    # _?_Brigham & Womens Hosp, Ctr Expt Therapeut & Reperfus Injury, Boston, MA 02115 USA|
    # Harvard Univ, Sch Med, Dept Oral Med Infect & Immun, Harvard Sch Dent Med, Boston, MA 02115 USA|
    # Kings Coll London, Div Cardiovasc, London WC2R 2LS, England|
    # UCL, Ctr Clin Pharmacol & Therapeut, Div Med, London, England|
    # Univ Edinburgh, MRC Ctr Inflammat Res, Queens Med Res Inst, Edinburgh, Midlothian, Scotland|
    # Trinity Coll Dublin, Sch Biochem & Immunol, Dublin, Ireland|
    # Queen Mary Univ London, William Harvey Res Inst, London, England|
    # Univ Calgary, Inflammat Res Network, Calgary, AB, Canada||
    # Univ Edinburgh, Queens Med Res Inst, Ctr Inflammat Res, MRC, Edinburgh, Midlothian, Scotland||
    # Univ Edinburgh, Queens Med Res Inst, Med Res Council Ctr Inflammat Res, Edinburgh, Midlothian, Scotland||
    # Univ Edinburgh, MRC, Queens Med Res Inst, Ctr Inflammat Res, Edinburgh, Midlothian, Scotland||
    # _?_Univ Modena, Dipartimento Med & Special Med, Div Med 3, Policlin, I-41100 Modena, Italy
 
   
   
           
    string= string.replace("||","|").replace("_?_","").replace("?","").replace("||","|").strip("|").strip().title()
         
        
#     print    
#     print string
        
#     if "Dept " in string or "Sch" in string  or "Coll" in string or "Inst" in string or "Hosp" in string:
                 
    string=string.replace("Dept ","Department ")
    string=string.replace("Dept,","Department,")
    #new_string=new_string.replace("Med,","Medicine,")
    string=string.replace("Sch ","School ")
    string=string.replace("Sch,","School,")
    string=string.replace("Coll ","College ")
    string=string.replace("Coll,","College, ")
    string=string.replace("Hosp,","Hospital,")
    string=string.replace("Hosp ","Hospital ")
    string=string.replace("Inst,","Institute,")
    string=string.replace("Inst ","Institute ")
    string=string.replace("Ctr ","Center ")
    string=string.replace("Ctr,","Center,")            
    
    
#     print "\n\nORIGINAL:",string  
          
            
            
    #     ['Osped S Carlo, UO Oncol Med, Potenza, Italy', 'Osped Civile, UO Oncol Med, Nola, Na, Italy', 
    #      'Univ Edinburgh, Queens Med Res Institute, Center Inflammat Res, Edinburgh, Midlothian, Scotland', 
    #      'MRC, Edinburgh, Midlothian, Scotland', 'Univ Edinburgh, School Med, MRC Center Inflammat Res, Queens Med Res Institute, Edinburgh EH16 4TJ, Midlothian, Scotland', 
    #      'Univ Highlands and Islands, Inverness IV2 3BL, Scotland', 'OSPED MAGGIORE CA PIZZARDI,ENTE OSPED REG,SERV FIS SANITARIA,BOLOGNA,ITALY', 
    #      'Univ Edinburgh, Edinburgh EH8 9YL, Midlothian, Scotland', 'Univ Edinburgh, School Med, Queens Med Res Institute, Center Inflammat Res,MRC, Edinburgh, Midlothian, Scotland']

    
    
    new_string_list= string.split("|")              

#     print "LISTA:",new_string_list

    flag_return=0
    lista_dept=[]
    for i in range(len(new_string_list)):
        
        try:
            dept=new_string_list[i].split(",")[1].strip()
        except IndexError:
            dept=""
            print new_string_list[i]
            #raw_input()
       
        if "Department" in dept: 
           lista_dept.append(dept)
           flag_return=1 
        elif "Sch" in dept  :
            lista_dept.append(dept)
            flag_return=1
        elif "Coll" in dept :
            lista_dept.append(dept)
            flag_return=1#print df_disamb_wos_test['author_names'].iloc[10]    # list with one single element, including all name spellings for an author:    ['Fransen, C.|Fransen, Ch|Fransen, C']
    
        elif "Inst" in dept :
            lista_dept.append(dept)
            flag_return=1
        elif "Hosp" in dept  :
            lista_dept.append(dept)
            flag_return=1
        elif "Lab" in dept :
            lista_dept.append(dept)
            flag_return=1
        elif "Center" in dept:
            lista_dept.append(dept)
            flag_return=1

    
    if flag_return >0:
#         print  "-----------",list(set(lista_dept))  # i get all the unique dept   
#         raw_input()
        return list(set(lista_dept))
    else:
        #print None
        return None

    
    
    
    
    
########################
#####################

#print df_disamb_wos_test['author_names'].iloc[10]    # list with one single element, including all name spellings for an author:    ['Fransen, C.|Fransen, Ch|Fransen, C']
# for item in df_disamb_wos_test['author_names'].iloc[10]:
#     print item
    


def fix_university_names(string):
  
#     print "ORIGINAL:  ",string

    #new_string= string.replace("||?||","").replace("||","\r\n ").replace("|","\r\n ").replace("_?_","")
    new_string= string.replace("||","|").replace("_?_","").replace("?","").replace("||","|").strip("|").title()
    
      
    new_string=new_string.replace("Mit","Massachusetts Institute of Technology")
    new_string=new_string.replace("Georgia Inst Technol","Georgia Institute of Technology")
    new_string=new_string.replace("Penn State","Pennsylvania State")
    new_string=new_string.replace("Univ Calif Berkeley","University of California Berkeley")
    new_string=new_string.replace("Univ Calif Los Angeles","University of California Los Angeles")
    new_string=new_string.replace("Univ Calif Davis","University of California Davis")
    new_string=new_string.replace("Univ Calif San Diego","University of California San Diego")
    new_string=new_string.replace("Penn State Univ,","Pennsylvania State University,")
    new_string=new_string.replace("Univ So Calif","University of Southern California")
    new_string=new_string.replace("NYU","New York University")





    new_string=new_string.replace('Univ,','University,')
    new_string=new_string.replace('Univ ','University ')
    
    new_string=new_string.replace('Natl,','National,')
    new_string=new_string.replace('Natl ','National ')
    
    new_string=new_string.replace('Hosp,','Hospital,')
    new_string=new_string.replace('Hosp ','Hospital ')
   
    new_string=new_string.replace("Sch ","School ")
    new_string=new_string.replace("Sch,","School,")
    
    new_string=new_string.replace("Coll ","College ")
    new_string=new_string.replace("Coll,","College, ")

    new_string=new_string.replace("Inst,","Institute,")
    new_string=new_string.replace("Inst ","Institute ")
    
    new_string=new_string.replace("Ctr ","Center ")
    new_string=new_string.replace("Ctr,","Center,")  



    
    new_string=new_string.strip()
    
    new_string_list= new_string.split("|")
    for i in range(len(new_string_list)):
        new_string_list[i]=new_string_list[i].split(",")[0]
   
#     print          
#     print "AFTER:   ",list(set(new_string_list))
#     print 
#     raw_input()#print df_disamb_wos_test['author_names'].iloc[10]    # list with one single element, including all name spellings for an author:    ['Fransen, C.|Fransen, Ch|Fransen, C']
# for item in df_disamb_wos_test['author_names'].iloc[10]:
#     print item
    
    
    
    
    
    return list(set(new_string_list))
 
 
 

In [28]:
def get_first_name_initial(string):    # example of string:      'Rossi, Adriano G.|Rossi, Adriano|Rossi, A. G.|Rossi, A|Rossi, AG|ROSSI, AG|Rossi, A.|ROSSI, A'
    
    if "," not in string:       
        if " " in string:            
            string = string.replace(" ",", ")   # chinese people at NICO told me that most names without a comma look like they have the format: LAST FIRST
      
        
    
    try:                              
        new_string = string.split("|")[0].split(",")[1].replace(".","").strip().lower()        
        return new_string[0]       
    except:
        return None
    

In [29]:
def get_first_name_or_initial(string):    # example of string:   'Rossi, Adriano G.|Rossi, Adriano|Rossi, A. G.|Rossi, A|Rossi, AG|ROSSI, AG|Rossi, A.|ROSSI, A'
    
    try:
        #print string            
        
        lista_candidatos=[]
        
        for item in string.split("|"):
            
            if "," not in item:       
                if " " in item:            
                    item=item.replace(" ",", ")   # chinese people at NICO told me that most names without a comma look like they have the format: LAST FIRST
                                 
            lista_candidatos.append(item.replace(".","").split(",")[1].strip().lower())
        
        new_string = max(lista_candidatos, key=len)  # ipick the longes first name 
       
        return new_string
    
    except :
        return None

In [30]:
def get_middle_initial(string):    # example of string:   'Rossi, Adriano G.|Rossi, Adriano|Rossi, A. G.|Rossi, A|Rossi, AG|ROSSI, AG|Rossi, A.|ROSSI, A'
    
   
           
        
    new_string=None
        
    #try:
    lista_candidatos=[""]

    for item in string.split("|"):

        if "," not in item:       
            if " " in item:            
                item=item.replace(" ",", ")   # chinese people at NICO told me that most names without a comma look like they have the format: LAST FIRST
        try:                     
            lista_candidatos.append(item.replace(".","").split(",")[1].strip().lower().split(" ")[1]   )

        except IndexError: pass


    try:
        new_string = max(lista_candidatos, key=len)  # ipick the longes first name 

        return new_string
    except TypeError:pass

   # except  TypeError: pass      # for the cases where there is no info   
   
       
 


In [31]:
def get_only_firstname(string):     #   'Rossi, Adriano G.|Rossi, Adriano|Rossi, A. G.|Rossi, A|Rossi, AG|ROSSI, AG|Rossi, A.|ROSSI, A'
   
       
        
    lista_candidatos=[""]

    for item in string.split("|"):

        if "," not in item:       
            if " " in item:            
                item=item.replace(" ",", ")   # chinese people at NICO told me that most names without a comma look like they have the format: LAST FIRST


        try:
            lista_candidatos.append(item.replace(".","").split(",")[1].strip().lower().split(" ")[0]    )
        except IndexError: pass
    try:
        new_string = max(lista_candidatos, key=len)  # ipick the longes first name 

        return new_string
    except TypeError:pass
   


SyntaxError: invalid syntax (<ipython-input-31-e15de9cfb912>, line 22)

In [32]:
def get_lastname(string):
  
      

    lista_candidatos=[""]

    for item in string.split("|"):

        if "," not in item:       
            if " " in item:            
                item=item.replace(" ",", ")   # chinese people at NICO told me that most names without a comma look like they have the format: LAST FIRST

        lista_candidatos.append(item.replace(".","").split(",")[0].strip().lower())

    try:
        new_string = max(lista_candidatos, key=len)  # ipick the longes first name 

        return new_string
    
    except TypeError: pass
    
   

In [ ]:
#df_disamb_wos_test.shape

In [33]:
      
# FALTA:      "España - Wikimedia Commons".replace("ñ","n")

 
   

# ###### i add  new columns with processed info: 
## example (it is a string):  'Rossi, Adriano G.|Rossi, Adriano|Rossi, A. G.|Rossi, A|Rossi, AG|ROSSI, AG|Rossi, A.|ROSSI, A'
    

 
    

%time df_disamb_wos_test['Department'] = df_disamb_wos_test.affiliations.apply(get_department_names)

print "Department done"     
   
    

df_disamb_wos_test['firstname'] = df_disamb_wos_test.author_names.apply(get_only_firstname)

print "only firstname done"                
    
    
df_disamb_wos_test['firstname_initial'] = df_disamb_wos_test.author_names.apply(get_first_name_initial)
df_disamb_wos_test['middle'] = df_disamb_wos_test.author_names.apply(get_middle_initial)


print "first and middle initial done"          
     

df_disamb_wos_test['lastname'] = df_disamb_wos_test.author_names.apply(get_lastname)

print "lastnames done"   




    
    
    
df_disamb_wos_test['list_author_names'] = df_disamb_wos_test.apply(lambda row: list(set(row.author_names.lower().replace(".|","|").replace(". ",".").replace("."," ").split("|"))), axis=1)

print "list_author_names done"     
        



 
%time df_disamb_wos_test['University'] = df_disamb_wos_test.affiliations.apply(fix_university_names)
 
print "University done"      
    

   
   

df_disamb_wos_test['list_years'] = df_disamb_wos_test.apply(lambda row: sorted(list(set(row.year.split("|")))), axis=1)

print "years done"     
   

# df_disamb_wos_test.firstname = df_disamb_wos_test.firstname.fillna('')  # i convert NaN into empty strings, so i can concatenate them without errors
# df_disamb_wos_test.lastname = df_disamb_wos_test.lastname.fillna('')


df_disamb_wos_test['full_name'] = df_disamb_wos_test['firstname'] +" "+ df_disamb_wos_test['middle']+" "+ df_disamb_wos_test['lastname']  #harold y hwang          ###   rick john w       ###   yanagisako sylvia
df_disamb_wos_test['full_name'] = df_disamb_wos_test.apply(lambda row: row.full_name.replace("  "," "), axis=1)





print "full name done" 



Icm


KeyboardInterrupt: 

In [ ]:


#df_disamb_wos_test.head(30)
#df_disamb_wos_test.middle_initial.unique()

# print df_disamb_wos_test.iloc[792]


In [ ]:
# print df_disamb_wos_test.author_names.iloc[434],df_disamb_wos_test.firstname.iloc[434]
# print df_disamb_wos_test.list_author_names.iloc[434]


# print df_disamb_wos_test.author_names.iloc[0]
# print 
# print df_disamb_wos_test.affiliations.iloc[0]

In [ ]:
#df_disamb_wos_test.head(3)

# df_disamb_wos_test.list_author_names.iloc[0]

In [ ]:
path='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/WoS_data/Disambiguated_authors/wos_author_disambiguated_2000-2015_USA_processed.pickle'
#DataFrame.to_pickle(path)[source]

%time df_disamb_wos_test.to_pickle(path)
#df_disamb_wos_test.shape
print "pickle done"



%time df_disamb_wos_test.to_csv(path.split(".pickle")[0]+".tsv", sep='\t', encoding='utf-8')#, columns = list_headers)
print "csv done"





In [ ]:
# ", 'Jetzt', 'Ponmudi', 'Nussbaum', 'Psarros', 
# #type(df_disamb_wos_test.University.iloc[0]))



# list_unique_lastnames=list(set(df_disamb_wos_test.lastname.unique()))
# # print list_unique_lastnames


# filename_pickle="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Merged_LinkedIn_WoS/list_unique_lastnames.pickle"
# %time pickle.dump(list_unique_lastnames, open(filename_pickle, 'wb'))




In [ ]:
# len(list_unique_lastnames)    #414123

# for unique_lastname in tqdm_notebook(list_unique_lastnames):

#     aux_df=df_disamb_wos_test[ df_disamb_wos_test.lastname == unique_lastname]

#     path='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/WoS_data/Disambiguated_authors/By_lastname/wos_author_disambiguated_2010-2015_USA_'+unique_lastname+'.pickle'

#     aux_df.to_pickle(path)
    
# print "/ndone"


# filename_pickle="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Merged_LinkedIn_WoS/list_unique_lastnames.pickle"
# %time pickle.dump(list_unique_lastnames, open(filename_pickle, 'wb'))

# import time
 
# # Wait for 5 seconds
# for i in range(10):
#     time.sleep(50)
#     localtime = time.localtime(time.time())
#     print "Local current time :", localtime
    
    
# print "done"

